In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import cv2
import os

import warnings
warnings.simplefilter('ignore')

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense,Input,Conv2D,MaxPool2D,Dropout
from tensorflow.keras.applications import VGG16,VGG19
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
print(len(os.listdir("/content/drive/MyDrive/Pleural Effusion/Train/Normal")))
print(len(os.listdir("/content/drive/MyDrive/Pleural Effusion/Train/Pleural")))

936
906


In [3]:
Train = "/content/drive/MyDrive/Pleural Effusion/Train/"

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.05,
                                   width_shift_range = 0.2,
                                   height_shift_range= 0.2,
                                   zoom_range=0.2,
                                   shear_range=0.2)

train_gen = train_datagen.flow_from_directory(
    Train,
    target_size=(224, 224),
    batch_size=16,
    subset='training',
    class_mode='binary')

val_gen = train_datagen.flow_from_directory(
    Train, 
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    subset='validation')

Found 1751 images belonging to 2 classes.
Found 91 images belonging to 2 classes.


In [7]:
resnet_layer = tf.keras.applications.ResNet152V2(input_shape=(224,224,3),include_top=False,weights="imagenet",
                                                    pooling='avg')

for layers in resnet_layer.layers:
    layers.trainable=False
    
output_layer = Dense(1,activation='sigmoid')(resnet_layer.output)

resnet_model = Model(resnet_layer.input,output_layer)

In [8]:
resnet_model.compile(optimizer=tf.keras.optimizers.Adam(beta_1=0.9,beta_2=0.999,lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
resnet_history = resnet_model.fit(train_gen,validation_data=val_gen,epochs=15,steps_per_epoch=100,batch_size=16)

Epoch 1/15
100/100 [==============================] - 1053s 10s/step - loss: 0.3869 - accuracy: 0.8200 - val_loss: 0.3333 - val_accuracy: 0.8681
Epoch 2/15
100/100 [==============================] - 139s 1s/step - loss: 0.2534 - accuracy: 0.9056 - val_loss: 0.3996 - val_accuracy: 0.8132
Epoch 3/15
100/100 [==============================] - 64s 640ms/step - loss: 0.2111 - accuracy: 0.9221 - val_loss: 0.2897 - val_accuracy: 0.8681
Epoch 4/15
100/100 [==============================] - 56s 557ms/step - loss: 0.1953 - accuracy: 0.9296 - val_loss: 0.3667 - val_accuracy: 0.8352
Epoch 5/15
100/100 [==============================] - 55s 545ms/step - loss: 0.1899 - accuracy: 0.9294 - val_loss: 0.3325 - val_accuracy: 0.8462
Epoch 6/15
100/100 [==============================] - 55s 547ms/step - loss: 0.1751 - accuracy: 0.9428 - val_loss: 0.3034 - val_accuracy: 0.8901
Epoch 7/15
100/100 [==============================] - 55s 545ms/step - loss: 0.1743 - accuracy: 0.9359 - val_loss: 0.3000 - val_accu

In [ ]:
resnet_model.save("/content/drive/MyDrive/Pleural Effusion/")